<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=472930392cb3451493f9e7c4a2c46d853e575b4209ceab37efa8cf7d67c017b4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-09 10:10:50
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.38 C
-------------------
Today PnL: -1.29 L (-0.93%)
Current PnL: -30.68 L (-19.82%)
CY Booked + Current PnL: -16.39 L (-10.59%)
-------------------
Total profit:  1.04 L
Total loss:  -31.72 L
-------------------
Total Booked + Current PnL: 10.59 L (8.28%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.39%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 95.31 L (69.27%)
Deployed:  1.28 C
Current:  1.38 C
CAGR/XIRR %: 4.13%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.40,-17.98,22.03,0.09,18210.0,-18116.0,82661.0,91.89,32.0,M-SC,2.19,253.0,-0.99,0.62,8.60,OX40N,NTT,DURABLES
43,ITC,452.00,-0.20,-1.90,12.58,10.44,24699.0,-3804.0,196334.0,-41.56,43.0,X-LC,1.66,5.0,-0.15,1.48,3.56,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,-1.04,-15.37,18.19,0.02,26870.0,-26838.0,147721.0,125.45,45.0,M-SC,16.70,234.0,-1.00,1.11,28.49,OX40N,NTT,PAINTS
50,MASFIN,398.61,-0.79,-5.51,29.17,22.05,27006.0,-5400.0,92580.0,-18.70,48.0,H-SC,6.89,164.0,-0.20,0.70,34.57,XR,ATH,FINANCE
35,ICICIGI,2252.93,-1.53,3.02,16.67,20.19,29936.0,5263.0,179583.0,-19.32,38.0,X-MC,5.93,68.0,0.18,1.35,18.59,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAPPSTMNDS,1480.71,0.65,-38.22,199.22,84.85,157734.0,-48989.0,79176.0,-26.54,44.0,H-SC,13.66,147.0,-0.31,0.60,1.62,AR,ATH,IT
11,BATAINDIA,2096.00,0.57,-38.46,119.72,35.20,94792.0,-49492.0,79178.0,3.83,26.0,X-SC,15.30,92.0,-0.52,0.60,0.57,X40,NTT,FOOTWEAR
42,IRCTC,1269.00,0.52,-19.89,90.90,52.93,196988.0,-53800.0,216708.0,-40.68,31.0,H-MC,5.43,118.0,-0.27,1.63,0.52,XY24,BTT,TRAVEL
36,ICICIPRULI,790.00,0.49,2.66,27.57,30.97,50023.0,4702.0,181440.0,-21.49,52.0,X-MC,2.02,75.0,0.09,1.37,15.56,X40,ATH,INSURANCE
34,HONAUT,58357.33,0.34,-15.60,69.82,43.32,95975.0,-25412.0,137460.0,-27.45,39.0,X-SC,3.74,90.0,-0.26,1.04,5.54,X40N,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-4.73,-48.60,163.16,35.27,95233.0,-55181.0,58368.0,-59.76,18.0,L-SC,12.34,270.0,-0.58,0.44,42.86,XR,NTT,HOTELS
5,ASIANPAINT,3460.25,-3.89,-1.63,22.95,20.95,56838.0,-4110.0,247658.0,-2.60,51.0,X-LC,19.05,31.0,-0.07,1.87,33.42,X40,ATH,PAINTS
28,GREENPANEL,537.00,-3.20,-39.97,138.43,43.14,152144.0,-73171.0,109907.0,146.12,21.0,M-SC,8.68,240.0,-0.48,0.83,3.38,XY24,NTT,MISC
44,JCHAC,2282.00,-2.77,-40.56,68.23,-0.01,51830.0,-51841.0,75964.0,15443.48,5.0,M-SC,3.50,233.0,-1.00,0.57,0.00,OX40N,NTT,AC
6,ASIANTILES,137.00,-2.55,-21.51,131.46,81.67,96797.0,-20178.0,73632.0,6611.11,28.0,L-SC,14.75,271.0,-0.21,0.56,42.59,XR,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.62,-9.73,130.31,107.91,173682.0,-14360.0,133284.0,-26.63,23.0,M-SC,11.95,216.0,-0.08,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.04,-15.37,18.19,0.02,26870.0,-26838.0,147721.0,125.45,45.0,M-SC,16.70,234.0,-1.00,1.11,28.49,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.40,-17.98,22.03,0.09,18210.0,-18116.0,82661.0,91.89,32.0,M-SC,2.19,253.0,-0.99,0.62,8.60,OX40N,NTT,DURABLES
66,SIS,528.00,-0.57,-25.60,62.99,21.26,52034.0,-28425.0,82607.0,1947.15,42.0,H-SC,4.65,166.0,-0.55,0.62,11.61,OX40N,NTT,MISC
47,KANSAINER,340.00,0.04,-25.93,53.20,13.47,106260.0,-69930.0,199737.0,-69.48,26.0,H-SC,3.22,158.0,-0.66,1.51,1.56,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.53,5.56,63.19,72.27,113029.0,9424.0,178872.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.35,12.08,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.53,5.56,63.19,72.27,113029.0,9424.0,178872.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.35,12.08,XR,NTT,IT
38,INDIAMART,4810.62,-1.13,-3.41,114.02,106.72,135830.0,-4208.0,119128.0,-52.15,32.0,H-SC,1.18,139.0,-0.03,0.90,19.17,AR,ATH,MISC
37,IEX,219.00,-0.16,-3.85,54.57,48.63,104916.0,-7694.0,192260.0,-36.00,49.0,H-SC,14.08,136.0,-0.07,1.45,7.26,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.74,-3.22,40.14,35.63,81418.0,-6740.0,202834.0,-16.74,34.0,H-MC,3.17,119.0,-0.08,1.53,12.92,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.62,-9.73,130.31,107.91,173682.0,-14360.0,133284.0,-26.63,23.0,M-SC,11.95,216.0,-0.08,1.00,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.51,-23.79,94.81,48.45,144869.0,-47707.0,152799.0,-29.32,10.0,X-MC,11.27,64.0,-0.33,1.15,0.00,X40N,ATH,FINANCE
22,DIXON,18931.72,-0.77,-13.56,40.66,21.58,49252.0,-19008.0,121131.0,-56.42,24.0,X-MC,5.47,56.0,-0.39,0.91,7.24,X40N,ATH,IT
46,JSWINFRA,342.00,-1.12,-12.40,31.54,15.23,55353.0,-24847.0,175500.0,-25.15,24.0,X-MC,7.96,66.0,-0.45,1.32,15.85,X40N,NTT,REALTY
75,TMPV,600.00,-0.93,-18.23,73.89,42.18,166997.0,-50402.0,226008.0,-26.74,25.0,X-LC,5.10,3.0,-0.30,1.70,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.57,-38.46,119.72,35.20,94792.0,-49492.0,79178.0,3.83,26.0,X-SC,15.30,92.0,-0.52,0.60,0.57,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-1.56,-21.92,96.60,53.50,227588.0,-66153.0,235598.0,-62.23,31.0,X-MC,1.25,58.0,-0.29,1.78,4.64,XY24,NTT,FMCG
43,ITC,452.0,-0.20,-1.90,12.58,10.44,24699.0,-3804.0,196334.0,-41.56,43.0,X-LC,1.66,5.0,-0.15,1.48,3.56,X40,NTT,FMCG
3,ACC,3906.0,-0.62,-24.99,118.84,64.15,212118.0,-59461.0,178490.0,-56.31,35.0,X-SC,1.79,82.0,-0.28,1.35,0.12,XY24,BTT,CEMENT
15,CAMS,4762.0,-1.20,-80.49,539.71,24.79,265160.0,-202726.0,49130.0,-81.32,38.0,X-SC,1.81,86.0,-0.76,0.37,20.11,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.49,2.66,27.57,30.97,50023.0,4702.0,181440.0,-21.49,52.0,X-MC,2.02,75.0,0.09,1.37,15.56,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.51,-23.79,94.81,48.45,144869.0,-47707.0,152799.0,-29.32,10.0,X-MC,11.27,64.0,-0.33,1.15,0.0,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-1.10,-10.04,40.14,26.07,59126.0,-16440.0,147300.0,-33.76,28.0,X-MC,11.52,55.0,-0.28,1.11,0.0,X40,ATH,APPARELS
59,RELAXO,1176.00,-0.50,-48.73,197.08,52.31,146667.0,-70740.0,74420.0,-45.41,30.0,X-SC,6.18,91.0,-0.48,0.56,0.0,X40N,NTT,FOOTWEAR
55,QUESS,424.00,-2.24,-32.58,111.87,42.84,49028.0,-21180.0,43826.0,-55.92,34.0,X-SC,37.14,83.0,-0.43,0.33,0.0,XY24,NTT,MISC
54,PGHH,17907.65,-1.30,-6.18,42.57,33.76,80208.0,-12405.0,188415.0,-33.85,33.0,X-MC,5.11,57.0,-0.15,1.42,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-2.24,-32.58,111.87,42.84,49028.0,-21180.0,43826.0,-55.92,34.0,X-SC,37.14,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-1.20,-80.49,539.71,24.79,265160.0,-202726.0,49130.0,-81.32,38.0,X-SC,1.81,86.0,-0.76,0.37,20.11,X40N,NTT,MISC
59,RELAXO,1176.00,-0.50,-48.73,197.08,52.31,146667.0,-70740.0,74420.0,-45.41,30.0,X-SC,6.18,91.0,-0.48,0.56,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.57,-38.46,119.72,35.20,94792.0,-49492.0,79178.0,3.83,26.0,X-SC,15.30,92.0,-0.52,0.60,0.57,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.77,-13.56,40.66,21.58,49252.0,-19008.0,121131.0,-56.42,24.0,X-MC,5.47,56.0,-0.39,0.91,7.24,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-1.61,-13.48,37.86,19.28,113324.0,-46632.0,299324.0,-24.60,59.0,X-LC,5.78,1.0,-0.41,2.26,10.66,X40,ATH,IT
41,INFY,2275.00,-1.32,8.21,43.13,54.87,146023.0,25676.0,338564.0,-15.65,61.0,X-LC,3.37,2.0,0.18,2.55,17.17,X40,BTT,IT
75,TMPV,600.00,-0.93,-18.23,73.89,42.18,166997.0,-50402.0,226008.0,-26.74,25.0,X-LC,5.10,3.0,-0.30,1.70,0.00,XY24,NTT,AUTO
81,VBL,671.64,-1.92,-6.92,45.76,35.67,134530.0,-21852.0,293990.0,-17.96,46.0,X-LC,4.86,4.0,-0.16,2.22,5.98,X40N,ATH,FMCG
43,ITC,452.00,-0.20,-1.90,12.58,10.44,24699.0,-3804.0,196334.0,-41.56,43.0,X-LC,1.66,5.0,-0.15,1.48,3.56,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,-0.79,-5.51,29.17,22.05,27006.0,-5400.0,92580.0,-18.70,48.0,H-SC,6.89,164.0,-0.20,0.70,34.57,XR,ATH,FINANCE
13,BSOFT,831.70,-0.86,-23.05,95.95,50.79,103851.0,-32418.0,108235.0,-1.43,67.0,H-SC,5.79,151.0,-0.31,0.82,26.02,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.04,-15.37,18.19,0.02,26870.0,-26838.0,147721.0,125.45,45.0,M-SC,16.70,234.0,-1.00,1.11,28.49,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-3.89,-1.63,22.95,20.95,56838.0,-4110.0,247658.0,-2.60,51.0,X-LC,19.05,31.0,-0.07,1.87,33.42,X40,ATH,PAINTS
68,STARHEALTH,761.00,-0.16,-10.80,65.09,47.26,163218.0,-30366.0,250757.0,21.68,34.0,H-SC,13.17,171.0,-0.19,1.89,34.35,XY24,NTT,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.86,-23.05,95.95,50.79,103851.0,-32418.0,108235.0,-1.43,67.0,H-SC,5.79,151.0,-0.31,0.82,26.02,XR,ATH,IT
32,HCLTECH,1922.01,-1.27,8.19,15.29,24.73,40019.0,19811.0,261735.0,12.76,64.0,X-LC,5.77,7.0,0.50,1.97,24.67,X40,ATH,IT
84,WIPRO,420.00,-1.53,5.56,63.19,72.27,113029.0,9424.0,178872.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.35,12.08,XR,NTT,IT
5,ASIANPAINT,3460.25,-3.89,-1.63,22.95,20.95,56838.0,-4110.0,247658.0,-2.60,51.0,X-LC,19.05,31.0,-0.07,1.87,33.42,X40,ATH,PAINTS
73,TCS,4389.97,-1.61,-13.48,37.86,19.28,113324.0,-46632.0,299324.0,-24.60,59.0,X-LC,5.78,1.0,-0.41,2.26,10.66,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.85
1,25,45.87
2,50,76.78


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.76
MC    30.37
LC    24.86
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.24
X40      22.62
X40N     10.86
XR       10.00
XY25      9.45
AR        9.36
OX40N     7.69
SR        1.02
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.40
X-MC    23.62
X-LC    21.27
M-SC    12.00
X-SC     7.00
H-MC     4.98
M-MC     1.41
L-SC     1.36
M-LC     1.35
H-LC     1.19
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.51,-8.41,43.46
IT,12.91,-17.79,81.47
FINANCE,9.47,-23.43,75.59
ELECTRICAL,6.27,-12.78,54.09
MISC,6.12,-59.80,125.27
PAINTS,6.11,-14.01,30.94
INSURANCE,4.61,-3.33,39.75
PHARMA,4.12,-2.11,34.79
AUTO,3.42,-26.26,80.40


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3289050.0,21
XR,1337607.0,13
AR,1330540.0,10
X40,1036451.0,14
X40N,1007322.0,9
OX40N,770710.0,10
XY25,388832.0,6
SR,285749.0,2
MH,85036.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3745118.0,25
M-SC,1464071.0,15
X-MC,1346921.0,16
X-SC,991750.0,8
X-LC,942133.0,11
H-MC,417316.0,3
L-SC,281400.0,3
M-LC,113029.0,1
H-LC,75773.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1317164.0      6
           AR         905783.0      5
M-SC       XY24       827810.0      6
H-SC       XR         798984.0      7
X-SC       X40N       507802.0      3
X-MC       X40        467721.0      7
           XY24       401880.0      3
X-LC       X40        380903.0      5
H-SC       OX40N      352402.0      4
M-SC       OX40N      328938.0      5
X-SC       XY24       296121.0      3
X-MC       X40N       287972.0      4
H-SC       SR         285749.0      2
X-LC       XY24       249087.0      2
H-MC       AR         220328.0      2
X-LC       X40N       211548.0      2
H-MC       XY24       196988.0      1
L-SC       XR         192030.0      2
X-MC       XY25       189348.0      2
X-SC       X40        187827.0      2
M-SC       XR         178667.0      2
           AR         128656.0      2
M-LC       XR         113029.0      1
X-LC       XY25       100595.0      2
L-SC       OX40N       89370.0      1
H-SC       MH          85036.0      1
H-LC       AR          75773.0      1
M-MC       XY25        58249.0      1
L-MC       XR          54897.0      1
L-LC       XY25        40640.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
